# Tải video YouTube kèm phụ đề (yt-dlp)

Mục tiêu: tạo công cụ tải video từ YouTube, kèm phụ đề (subtitles hoặc auto-captions), lưu vào thư mục dự án `API/AI/data/videos/`.

Thành phần:
- Cài đặt `yt-dlp` và kiểm tra `ffmpeg`.
- Hàm `download_youtube_with_subtitles(...)` tải video/playlist, ưu tiên MP4 và thu thập phụ đề.
- Tuỳ chọn ngôn ngữ phụ đề (ví dụ `vie`, `eng`), định dạng `.srt`.
- Log tiến độ và trả về danh sách đường dẫn tệp đã tải.

**Lưu ý**: `ffmpeg` cần có trong PATH để hợp nhất video/audio và chuyển định dạng phụ đề.

In [1]:

# Cài đặt thư viện cần thiết
# - yt-dlp: công cụ tải YouTube
# - pathlib: làm việc với đường dẫn
# Gợi ý cài ffmpeg trước (Windows có thể dùng winget hoặc tải binary).

# !pip install --upgrade yt-dlp

import os, sys, shutil
from pathlib import Path

# Kiểm tra ffmpeg đã có trong PATH hay chưa
# Nếu None -> cần cài ffmpeg để ghép video/audio và chuyển phụ đề
ffmpeg_path = shutil.which("ffmpeg")
print("ffmpeg:", ffmpeg_path if ffmpeg_path else "CHƯA PHÁT HIỆN (cần cài)")

# Tạo thư mục đích theo chuẩn OLMS
data_root = Path("API/AI/data/videos")
data_root.mkdir(parents=True, exist_ok=True)
print("Thư mục lưu:", data_root.resolve())


   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ------ --------------------------------- 0.5/3.2 MB 4.2 MB/s eta 0:00:01
   ------------------- -------------------- 1.6/3.2 MB 4.7 MB/s eta 0:00:01
   ----------------------------------- ---- 2.9/3.2 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 4.5 MB/s eta 0:00:00
ffmpeg: D:\ProgramFiles\ffmpeg-8.0-essentials_build\bin\ffmpeg.EXE
Thư mục lưu: D:\Projects\Basic_Project_CSharp\Online_Management_System - Copy\API\AI\API\AI\data\videos



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from typing import List, Dict, Any, Optional
from yt_dlp import YoutubeDL
from pathlib import Path
import os, shutil

def download_youtube_with_subtitles(
    url: str,
    out_dir: str = "API/AI/data/videos",
    prefer_mp4: bool = True,
    subtitles_langs: Optional[List[str]] = None,
    use_auto_captions: bool = True,
    subtitle_format: str = "srt"
) -> List[str]:
    """
    Tải video/playlist YouTube về thư mục out_dir và tải phụ đề nếu có.
    - Ưu tiên định dạng MP4; nếu không có thì lấy best và hợp nhất.
    - Phụ đề: ưu tiên ngôn ngữ chỉ định (subtitles_langs). Nếu không có, có thể dùng auto-captions.
    - Định dạng phụ đề: srt (khuyến nghị, cần ffmpeg).

    Trả về: danh sách đường dẫn file video/phụ đề đã tải.
    """

    # Chuẩn hoá danh sách ngôn ngữ phụ đề; hỗ trợ cả "vi/vie" và "en/eng"
    if subtitles_langs is None:
        subtitles_langs = ["vi", "vie", "en", "eng"]

    # Đảm bảo thư mục đích tồn tại
    out_dir_path = Path(out_dir)
    out_dir_path.mkdir(parents=True, exist_ok=True)

    # Mẫu tên đầu ra: <out_dir>/<video_id>-<title>.<ext> để tránh trùng tiêu đề
    out_tmpl = str(out_dir_path / "%(id)s-%(title)s.%(ext)s")

    # Chọn format ưu tiên MP4
    if prefer_mp4:
        # best mp4 video + best m4a audio, fallback best[ext=mp4], cuối cùng best
        fmt = "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best"
        merge_to = "mp4"
    else:
        fmt = "bv*+ba/best"
        merge_to = "mp4"

    # Hook tiến độ: in phần trăm và tốc độ
    def _progress_hook(d: Dict[str, Any]):
        if d.get("status") == "downloading":
            percent = d.get("_percent_str", "").strip()
            speed = d.get("speed")
            speed_str = f"{speed/1024/1024:.2f} MB/s" if speed else "? MB/s"
            fname = d.get("filename", "")
            print(f"[DL] {percent} {speed_str}  {fname}", end="\r")
        elif d.get("status") == "finished":
            fname = d.get("filename", "")
            print(f"\n[MERGE] Hoàn tất đoạn: {fname} -> hợp nhất nếu cần...")

    # Tuỳ chọn yt-dlp: tải video + phụ đề và chuyển sang srt
    ydl_opts = {
        "outtmpl": out_tmpl,                   # mẫu đặt tên file
        "format": fmt,                         # chọn chất lượng/định dạng
        "merge_output_format": merge_to,       # định dạng sau hợp nhất
        "noplaylist": False,                   # cho phép playlist
        "ignoreerrors": True,                  # bỏ qua video lỗi trong playlist
        "progress_hooks": [_progress_hook],    # log tiến độ
        "quiet": False,
        "no_warnings": True,

        # Phụ đề
        "writesubtitles": True,                # tải phụ đề uploader
        "writeautomaticsub": use_auto_captions,# bật auto-captions nếu cần
        "subtitleslangs": subtitles_langs,     # ưu tiên ngôn ngữ
        "subtitlesformat": subtitle_format,    # "srt" hoặc "vtt"

        # Chuyển phụ đề sang định dạng đích (srt cần ffmpeg)
        "postprocessors": [
            {"key": "FFmpegSubtitlesConvertor", "format": subtitle_format}
        ],
    }

    # Thu thập tệp đầu ra bằng cách:
    # - Lấy thông tin từ extract_info(download=True)
    # - Quét thư mục out_dir theo id để gom video/subtitle tương ứng
    downloaded_files: List[str] = []

    def _collect_entry_outputs(entry: Dict[str, Any]):
        """
        Gom tệp của một entry (video đơn trong playlist hoặc video đơn lẻ).
        - Ưu tiên đọc 'requested_downloads' nếu có 'filepath'.
        - Bổ sung quét thư mục theo id để lấy phụ đề .srt/.vtt và video đã ghép.
        """
        vid = entry.get("id")
        if not vid:
            return
        # 1) requested_downloads: nhiều item (video, audio, merged...). Lấy filepath nếu có.
        for rd in entry.get("requested_downloads", []) or []:
            fp = rd.get("filepath")
            if fp and Path(fp).exists():
                downloaded_files.append(str(Path(fp).resolve()))
        # 2) filepath trực tiếp ở entry (một số trường hợp)
        fp = entry.get("filepath")
        if fp and Path(fp).exists():
            downloaded_files.append(str(Path(fp).resolve()))
        # 3) Quét thư mục theo id để bắt cả phụ đề và video hợp nhất
        for ext in ["mp4", "mkv", "webm", "mov", "m4a", "mp3", subtitle_format, "vtt"]:
            for p in out_dir_path.glob(f"{vid}-*.{ext}"):
                downloaded_files.append(str(p.resolve()))

    # Thực thi tải: extract_info(download=True) trả về dict; nếu là playlist có 'entries'
    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)

    # Nếu playlist: duyệt entries; nếu video đơn: xử lý trực tiếp
    if info is None:
        print("\n[WARN] Không lấy được thông tin video.")
    elif "entries" in info and isinstance(info["entries"], list):
        for e in info["entries"]:
            if e:  # có thể None nếu ignoreerrors
                _collect_entry_outputs(e)
    else:
        _collect_entry_outputs(info)

    # Loại trùng và trả về
    uniq, seen = [], set()
    for f in downloaded_files:
        if f not in seen:
            seen.add(f)
            uniq.append(f)

    print(f"\n[OK] Tổng số tệp đã ghi: {len(uniq)} vào {Path(out_dir).resolve()}")
    for f in uniq:
        print(" -", f)
    return uniq


In [5]:

# Ví dụ sử dụng:
# - Tải 1 video đơn lẻ và phụ đề tiếng Việt/Anh nếu có.
# - Lưu về API/AI/data/videos
# - Trả về danh sách đường dẫn tệp (video .mp4 và phụ đề .srt)

# url_demo có thể thay bằng video bất kỳ cần thử
url_demo = "https://youtu.be/AWWtNNIfAOo?si=yEItsV2Z1mcHH64m"

files = download_youtube_with_subtitles(
    url=url_demo,
    out_dir="API/AI/data/videos",
    prefer_mp4=True,
    subtitles_langs=["vi","vie","en","eng"],
    use_auto_captions=True,
    subtitle_format="srt"
)

# In danh sách tệp đã lưu
from pprint import pprint
pprint(files)


[youtube] Extracting URL: https://youtu.be/AWWtNNIfAOo?si=yEItsV2Z1mcHH64m
[youtube] AWWtNNIfAOo: Downloading webpage
[youtube] AWWtNNIfAOo: Downloading tv client config
[youtube] AWWtNNIfAOo: Downloading tv player API JSON
[youtube] AWWtNNIfAOo: Downloading web safari player API JSON
[youtube] AWWtNNIfAOo: Downloading player 0004de42-main
[youtube] AWWtNNIfAOo: Downloading m3u8 information
[info] AWWtNNIfAOo: Downloading subtitles: vi, en
[info] AWWtNNIfAOo: Downloading 1 format(s): 136+140
[info] Writing video subtitles to: API\AI\data\videos\AWWtNNIfAOo-08： Lập trình C# cơ bản - Toán tử 3 ngôi C# - bài tập Csharp 10 - Tự học lập trình C# cho người mới.vi.srt
[download] Destination: API\AI\data\videos\AWWtNNIfAOo-08： Lập trình C# cơ bản - Toán tử 3 ngôi C# - bài tập Csharp 10 - Tự học lập trình C# cho người mới.vi.srt
[download] 100% of   17.48KiB in 00:00:00 at 92.95KiB/s0% 0.29 MB/s  API\AI\data\videos\AWWtNNIfAOo-08： Lập trình C# cơ bản - Toán tử 3 ngôi C# - bài tập Csharp 10 - Tự

In [ ]:

# Liệt kê toàn bộ tệp trong thư mục đích để kiểm tra
from pathlib import Path
from datetime import datetime

root = Path("API/AI/data/videos")
rows = []
for p in sorted(root.glob("*")):
    if p.is_file():
        st = p.stat()
        rows.append({
            "name": p.name,
            "size_MB": round(st.st_size/1024/1024, 2),
            "modified": datetime.fromtimestamp(st.st_mtime).strftime("%Y-%m-%d %H:%M:%S"),
            "path": str(p.resolve())
        })

import pandas as pd
pd.DataFrame(rows).head(50)
